In [1]:
import pandas as pd
import numpy as np
import os
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import json
import csv
from collections import defaultdict
import functools
import operator

In [2]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/")
f_files = []
m_files = []
male_file = "share_by_occupation_male.txt"
fem_file = "share_by_occupation_female.txt"

with open(male_file, "r") as f:
    lines = f.readlines()
    for l in lines:    
        file = l.rstrip("\n") + ".csv"
        m_files.append(file)
print(m_files)


with open(fem_file, "r") as f:
    lines = f.readlines()
    for l in lines:    
        file = l.rstrip("\n") + ".csv"
        f_files.append(file)
print(f_files)

['SEOMA0AB.csv', 'SEOMA15A.csv', 'SEOMA15B.csv', 'SEOMA2AB.csv', 'SEOMA3AB.csv', 'SEOMA4AB.csv', 'SEOMA5AB.csv', 'SEOMA6AB.csv', 'SEOMA7AB.csv', 'SEOMA8AB.csv', 'SEOMA9AB.csv']
['SEOFE0AB.csv', 'SEOFE15A.csv', 'SEOFE15B.csv', 'SEOFE2AB.csv', 'SEOFE3AB.csv', 'SEOFE4AB.csv', 'SEOFE5AB.csv', 'SEOFE6AB.csv', 'SEOFE7AB.csv', 'SEOFE8AB.csv', 'SEOFE9AB.csv']


In [4]:
print(len(f_files))

11


In [3]:
# Read csv files from No Ceilings project folder
path = "/Users/ericaxia/Downloads/Github/project-girlboss/data/noceilings-data-master/csv"
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))

# Narrow down to just the files I want
m_result = [f for f in result if f in m_files]
dfs_male = []
for f in m_result:
    df = pd.read_csv(f)
    dfs_male.append((f, df))    
print(f"Using {len(dfs_male)} csv files for male")

f_result = [f for f in result if f in f_files]
dfs_female = []
for f in f_result:
    df = pd.read_csv(f)
    dfs_female.append((f, df))    
print(f"Using {len(dfs_female)} csv files for female")

Using 11 csv files for male
Using 11 csv files for female


## Let's narrow down to a chosen year

In [4]:
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")

In [5]:
## NEW: map ISO -> Country name for better understandability in graph
with open('iso_to_country_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    country_names = {rows[0]: rows[1] for rows in reader}
    # print(country_names)

country_names_dict = { v: k for k, v in country_names.items()}
# country_names_dict


In [6]:
def convert_iso_to_name(iso):
    # print(x)
    if iso in country_names_dict:
        return country_names_dict[iso]
    else:
        return iso

In [7]:
shapes = []
def clean_dfs(dfs, gender, year):
    countries = []
    json_names = []
    for i in range(len(dfs)):
        name = dfs[i][0].strip(".csv")
        df = dfs[i][1]
        df = df[['ISO', year]]
        df.dropna(inplace=True)  # drop nans
        df.columns = ['name', 'value']  # rename cols
        # change ISO to full country
        df['name'] = df['name'].apply(lambda x: convert_iso_to_name(x))
        df['gender'] = gender  # set gender as new column
        # drop weird ones that dont match any full country name
        df = df[~df["name"].str.isupper()]
        shapes.append(df.shape)
        ## EXPORT
        # df.to_json(f"{name}.json", orient='records')  # export to json
        # print(f"Exported {name} data with shape {df.shape}")
        json_names.append(f"{name}.json")
        # # get unique countries
        names = list(df['name'].unique())
        countries.append(names)
    return json_names, countries

In [8]:
year = '2012'  # chosen year for plot
json_names_fem, countries_f = clean_dfs(dfs_female, 'female', year)
json_names_male, countries_m = clean_dfs(dfs_male, 'male', year)

/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/var/folders/mm/jp6_fd4x0g126l5m3dfnk4k80000gn/T/ipykernel_18158/3941417616.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = df['name'].apply(lambda x: convert_iso_to_name(x))
/var/folders/mm/jp6_fd4x0g126l5m3dfnk4k80000gn/T/ipykernel_18158/3941417616.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [11]:
c = 0
for tup in shapes:
    nrows = tup[0]
    c += nrows
print(c)

686


In [35]:
countries_ff = set(functools.reduce(operator.iconcat, countries_f, []))
countries_mm = set(functools.reduce(operator.iconcat, countries_m, []))
all_countries = countries_ff.union(countries_mm)
print(len(all_countries))
print(all_countries)


35
{'Bulgaria', 'Portugal', 'Iceland', 'Italy', 'Austria', 'Belgium', 'Luxembourg', 'Moldova', 'Cyprus', 'Turkey', 'Ireland', 'Croatia', 'Netherlands', 'Singapore', 'Spain', 'Romania', 'Poland', 'Hungary', 'Denmark', 'United Kingdom', 'Finland', 'Greece', 'Malta', 'Lithuania', 'Slovenia', 'Czech Republic', 'Germany', 'North Macedonia', 'Switzerland', 'Norway', 'Slovak Republic', 'Estonia', 'Sweden', 'France', 'Latvia'}


In [37]:
# with open('countries_2012.txt', "w") as f:
#     for c in all_countries:
#         f.write(c + "\n")

In [ ]:
# ## How many countries across all years and not just 2012?
# countries = set()
# for df in dfs_female:
#     name = df[0].strip(".csv")
#     df = df[1]
#     # df = df[['ISO', year]]
#     # df.dropna(inplace=True)  # drop nans
#     # df.columns = ['name', 'value']  # rename cols
#     # change ISO to full country
#     df['name'] = df['ISO'].apply(lambda x: convert_iso_to_name(x))
#     df_countries = (df['name'].unique())
#     for c in df_countries:
#         # remove non-countries
#         if not c.startswith("#") and not c.isupper():
#             countries.add(c)  
#             if c == "United States":
#                 print(df[df['name']==c])  
# print(len(countries)) # 120 countries across all years, w/o dropping any nans

# Ideal nested JSON format as final product
That we can plug right into the zoomable circle packing chart 
Let names = country or code code, values = percentage 

```
{
    "name": "education",
    "children": [
        {
            "name": "Primary School Education, Women age 25+",
            "children": [
                {
                    "value": 95,
                    "name": "USA"
                },
                ...
            ]
        },
        {
            "name": "Secondary School Education, Women age 25+",
            "children": [
                {
                    "value": 96,
                    "name": "USA"
                },
                ...
            ]
        },
        ...
    ]
}

```

In [ ]:
## continue where i left off

In [ ]:
# Read back in the json files saved
os.chdir("/Users/ericaxia/Downloads/Github/dsci554/554_Project_Code/Project/d3layout_data")
extension = 'json'
result = glob.glob('*.{}'.format(extension))
# Narrow down to just the files I want
f_result = [f for f in result if f in json_names_fem]
m_result = [f for f in result if f in json_names_male]

In [ ]:
# import csv file with full dataset name mappings
with open('data_names_male.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dataset_names_male = {rows[0]: rows[1].rstrip(", Male") for rows in reader}
    print(dataset_names_male)

In [ ]:
# import csv file with full dataset name mappings
with open('data_names.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dataset_names_female = {rows[0]: rows[1].rstrip(", Female") for rows in reader}
    print(dataset_names_female)

# Create big json file

In [ ]:
data_dict = defaultdict(list)
# female
for f in f_result:
    ds_code = f.split('.json')[0]
    ds_name = dataset_names_female[ds_code]
    # print(ds_name)
    with open(f, "r") as read_file:
        data = json.load(read_file)  # list of dicts
        data_dict[ds_name].append(data)
# print(len(data_dict))

In [ ]:
# male
for f in m_result:
    ds_code = f.split('.json')[0]
    ds_name = dataset_names_male[ds_code]
    print(ds_name)
    with open(f, "r") as read_file:
        data = json.load(read_file)  # list of dicts
        data_dict[ds_name].append(data)


In [ ]:
# for occupation, list of lists ([male list], [female list]) 
## flatten the list of lists
data_dict2 = defaultdict(list)
for occ, lol in data_dict.items():
    print(occ)
    if len(lol) == 2:
        data_dict2[occ] = lol[0] + lol[1]
    elif len(lol) == 1:
        data_dict2[occ] = lol

In [ ]:
export_file = { "name": "employment", "children": [] }

for occ, l in data_dict2.items():
    inner_dict = { "name": occ, "children": l }
    export_file["children"].append(inner_dict)


In [ ]:
print(len(export_file['children']))
print(type(export_file['children']))

In [ ]:
# with open("all_employment_data.json", "w") as write_file:
#     json.dump(export_file, write_file)